In [1]:
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()
# 2021.10.01 데이터 추출 완료 - 31140 데이터가 대부분 없음, InterfaceError: Python type ResultSet cannot be converted > updowncd 값 2에 의한 에러
cursor = mydb.cursor(prepared=True)
citycode = '33360'

query1 = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query1)

location_list1 = []

for a in cursor:
    location_list1.append(a)

now = datetime.now()
nowDate = now.strftime('%Y%m%d')

b_cnt = 0
for z in location_list1:
    if b_cnt == 1:
        break
        
    location = z
    query2 = ("select distinct routeid from public_busroute_info where citycode = '{}'".format(location[0]))
    cursor.execute(query2)

    location_list2 = []

    for b in cursor:
        location_list2.append(b)

    cnt = 1
    for i in location_list2:
        routeId = i

        characters = "(',')"

        join_data = ''.join( x for x in routeId if x not in characters)
        print("조회날짜 : {} | location : {} | cnt : {} | start {} public_busroute_info".format(nowDate,location[0],cnt,join_data))

        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteAcctoThrghSttnList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : location[0],
            quote_plus('pageNo') : '1',
            quote_plus('numOfRows') : '10000',
            quote_plus('routeId') : join_data
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],join_data))
            mydb.close()
            b_cnt += 1
            break
        if rows == []:
            print("location {} | {} 데이터가 없습니다.".format(location[0],join_data))
            cnt += 1
            continue
        cnt += 1
        for j in rows:
            item = j
            lat = item.findAll('gpslati')                
            lng = item.findAll('gpslong')
            nodeid = item.findAll('nodeid')
            nodenm = item.findAll('nodenm')
            nodeno = item.findAll('nodeno')
            if nodeno == []:
                nodeno = None
            else:
                nodeno = nodeno[0].text
            nodeord = item.findAll('nodeord')
            routeid = item.findAll('routeid')  
            updowncd = item.findAll('updowncd')
            
            if updowncd == []:
                updowncd = None
            elif updowncd[0].text == '0':
                updowncd = '상행'
            elif updowncd[0].text == '1':
                updowncd = '하행'
            elif updowncd[0].text == '2':
                updowncd = '2'

            sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd, citycode) VALUES (?,?,?,?,?,?,?,?,?)"
            val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, nodeno, nodeord[0].text, routeid[0].text, updowncd, location[0])

            cursor.execute(sql, val)
            mydb.commit()

조회날짜 : 20211002 | location : 33020 | cnt : 1 | start CHB272000005 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 2 | start CHB272000006 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 3 | start CHB272000008 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 4 | start CHB272000010 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 5 | start CHB272000017 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 6 | start CHB272000019 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 7 | start CHB272000020 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 8 | start CHB272000021 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 9 | start CHB272000023 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 10 | start CHB272000026 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 11 | start CHB272000028 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt 

조회날짜 : 20211002 | location : 33020 | cnt : 95 | start CHB272000253 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 96 | start CHB272000254 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 97 | start CHB272000255 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 98 | start CHB272000279 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 99 | start CHB272000280 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 100 | start CHB272000317 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 101 | start CHB272000319 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 102 | start CHB272000320 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 103 | start CHB272000321 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 104 | start CHB272000327 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 105 | start CHB272000328 public_busroute_info
조회날짜 : 20211002 | location

조회날짜 : 20211002 | location : 33020 | cnt : 189 | start CHB272000576 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 190 | start CHB272000577 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 191 | start CHB272000578 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 192 | start CHB272000581 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 193 | start CHB272000583 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 194 | start CHB272000586 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 195 | start CHB272000588 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 196 | start CHB272000590 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 197 | start CHB272000593 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 198 | start CHB272000594 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 199 | start CHB272000597 public_busroute_info
조회날짜 : 20211002 | loc

조회날짜 : 20211002 | location : 33020 | cnt : 282 | start CHB272000213 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 283 | start CHB272000216 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 284 | start CHB272000217 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 285 | start CHB272000218 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 286 | start CHB272000219 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 287 | start CHB272000220 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 288 | start CHB272000221 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 289 | start CHB272000226 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 290 | start CHB272000228 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 291 | start CHB272000230 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 292 | start CHB272000281 public_busroute_info
조회날짜 : 20211002 | loc

조회날짜 : 20211002 | location : 33020 | cnt : 376 | start CHB272000299 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 377 | start CHB272000421 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 378 | start CHB272000438 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 379 | start CHB272000477 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 380 | start CHB272000482 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 381 | start CHB272000489 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 382 | start CHB272000506 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 383 | start CHB272000507 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 384 | start CHB272000511 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 385 | start CHB272000535 public_busroute_info
조회날짜 : 20211002 | location : 33020 | cnt : 386 | start CHB272000536 public_busroute_info
조회날짜 : 20211002 | loc

...

조회날짜 : 20211002 | location : 33030 | cnt : 121 | start JCB273000218 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 122 | start JCB273000233 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 123 | start JCB273000236 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 124 | start JCB273000239 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 125 | start JCB273000240 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 126 | start JCB273000241 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 127 | start JCB273000242 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 128 | start JCB273000251 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 129 | start JCB273000252 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 130 | start JCB273000253 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 131 | start JCB273000264 public_busroute_info
조회날짜 : 20211002 | loc

조회날짜 : 20211002 | location : 33030 | cnt : 215 | start JCB273000174 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 216 | start JCB273000199 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 217 | start JCB273000200 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 218 | start JCB273000221 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 219 | start JCB273000229 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 220 | start JCB273000235 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 221 | start JCB273000263 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 222 | start JCB273000265 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 223 | start JCB273000095 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 224 | start JCB273000096 public_busroute_info
조회날짜 : 20211002 | location : 33030 | cnt : 225 | start JCB273000097 public_busroute_info
조회날짜 : 20211002 | loc

조회날짜 : 20211002 | location : 33320 | cnt : 51 | start BEB275000045 public_busroute_info
조회날짜 : 20211002 | location : 33320 | cnt : 52 | start BEB275000048 public_busroute_info
조회날짜 : 20211002 | location : 33320 | cnt : 53 | start BEB275000056 public_busroute_info
조회날짜 : 20211002 | location : 33320 | cnt : 54 | start BEB275000057 public_busroute_info
조회날짜 : 20211002 | location : 33320 | cnt : 55 | start BEB275000059 public_busroute_info
조회날짜 : 20211002 | location : 33320 | cnt : 56 | start BEB275000067 public_busroute_info
조회날짜 : 20211002 | location : 33320 | cnt : 57 | start BEB275000071 public_busroute_info
조회날짜 : 20211002 | location : 33320 | cnt : 58 | start BEB275000075 public_busroute_info
조회날짜 : 20211002 | location : 33320 | cnt : 59 | start BEB275000080 public_busroute_info
조회날짜 : 20211002 | location : 33320 | cnt : 60 | start BEB275000015 public_busroute_info
조회날짜 : 20211002 | location : 33320 | cnt : 61 | start BEB275000017 public_busroute_info
조회날짜 : 20211002 | location : 333

조회날짜 : 20211002 | location : 33330 | cnt : 58 | start OCB276000034 public_busroute_info
조회날짜 : 20211002 | location : 33330 | cnt : 59 | start OCB276000037 public_busroute_info
조회날짜 : 20211002 | location : 33330 | cnt : 60 | start OCB276000051 public_busroute_info
조회날짜 : 20211002 | location : 33330 | cnt : 61 | start OCB276000062 public_busroute_info
조회날짜 : 20211002 | location : 33330 | cnt : 62 | start OCB276000068 public_busroute_info
조회날짜 : 20211002 | location : 33330 | cnt : 63 | start OCB276000069 public_busroute_info
조회날짜 : 20211002 | location : 33330 | cnt : 64 | start OCB276000091 public_busroute_info
조회날짜 : 20211002 | location : 33330 | cnt : 65 | start OCB276000105 public_busroute_info
조회날짜 : 20211002 | location : 33330 | cnt : 66 | start OCB276000036 public_busroute_info
조회날짜 : 20211002 | location : 33330 | cnt : 67 | start OCB276000038 public_busroute_info
조회날짜 : 20211002 | location : 33330 | cnt : 68 | start OCB276000039 public_busroute_info
조회날짜 : 20211002 | location : 333

조회날짜 : 20211002 | location : 33340 | cnt : 60 | start TSB277000052 public_busroute_info
조회날짜 : 20211002 | location : 33340 | cnt : 61 | start TSB277000056 public_busroute_info
조회날짜 : 20211002 | location : 33340 | cnt : 62 | start TSB277000066 public_busroute_info
조회날짜 : 20211002 | location : 33340 | cnt : 63 | start TSB277000068 public_busroute_info
조회날짜 : 20211002 | location : 33340 | cnt : 64 | start TSB277000069 public_busroute_info
조회날짜 : 20211002 | location : 33340 | cnt : 65 | start TSB277000070 public_busroute_info
조회날짜 : 20211002 | location : 33340 | cnt : 66 | start TSB277000073 public_busroute_info
조회날짜 : 20211002 | location : 33340 | cnt : 67 | start TSB277000081 public_busroute_info
조회날짜 : 20211002 | location : 33340 | cnt : 68 | start TSB277000029 public_busroute_info
조회날짜 : 20211002 | location : 33340 | cnt : 69 | start TSB277000040 public_busroute_info
조회날짜 : 20211002 | location : 33340 | cnt : 70 | start TSB277000053 public_busroute_info
조회날짜 : 20211002 | location : 333